In [1]:
# load data

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from time import time

from data_util import get_data_set, maybe_download_and_extract
from model_util import model

In [3]:
train_x, train_y, train_l = get_data_set(cifar=10)
test_x, test_y, test_l = get_data_set("test", cifar=10)

x, y, output, global_step, y_pred_cls = model()

_IMG_SIZE = 32
_NUM_CHANNELS = 3
_BATCH_SIZE = 128
_CLASS_SIZE = 10
_ITERATION = 10000
_SAVE_PATH = "/tmp/tensorboard/cifar-10/"


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))


starter_learning_rate = 1e-4
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.96, staircase=True)

# Passing global_step to minimize() will increment it at each step.
optimizer = (
    tf.train.RMSPropOptimizer(learning_rate)
    .minimize(loss, global_step=global_step)
)
#optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-5).minimize(loss, global_step=global_step)


correct_prediction = tf.equal(y_pred_cls, tf.argmax(y, dimension=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("Loss/train", loss)
tf.summary.scalar("Accuracy/train", accuracy)



merged = tf.summary.merge_all()
saver = tf.train.Saver()
sess = tf.Session()
train_writer = tf.summary.FileWriter(_SAVE_PATH, sess.graph)



In [4]:
print(train_x.shape)
print(train_y.shape)
print(train_l[:5])
img_train_x = train_x.reshape(-1,32,32,3)
print(img_train_x.shape)

(50000, 3072)
(50000, 10)
['airplane', 'automobile', 'bird', 'cat', 'deer']
(50000, 32, 32, 3)


In [5]:
def train(num_iterations):
    '''
        Train CNN
    '''
    for i in range(num_iterations):
        randidx = np.random.randint(len(train_x), size=_BATCH_SIZE)
        batch_xs = train_x[randidx]
        batch_ys = train_y[randidx]

        start_time = time()
        i_global, _ = sess.run([global_step, optimizer], feed_dict={x: batch_xs, y: batch_ys})
        duration = time() - start_time

        if (i_global % 10 == 0) or (i == num_iterations - 1):
            _loss, batch_acc = sess.run([loss, accuracy], feed_dict={x: batch_xs, y: batch_ys})
            summary = tf.Summary(value=[
                tf.Summary.Value(tag="Loss/test", simple_value=_loss),
            ])
            train_writer.add_summary(summary, i_global)
            msg = "Global Step: {0:>6}, accuracy: {1:>6.1%}, loss = {2:.2f} ({3:.1f} examples/sec, {4:.2f} sec/batch)"
            print(msg.format(i_global, batch_acc, _loss, _BATCH_SIZE / duration, duration))

        if (i_global % 100 == 0) or (i == num_iterations - 1):
            
            
            data_merged, global_1 = sess.run([merged, global_step], feed_dict={x: batch_xs, y: batch_ys})
            acc = predict_test()

            summary = tf.Summary(value=[
                tf.Summary.Value(tag="Accuracy/test", simple_value=acc),
            ])
            train_writer.add_summary(data_merged, global_1)
            train_writer.add_summary(summary, global_1)

            saver.save(sess, save_path=_SAVE_PATH, global_step=global_step)
            print("Saved checkpoint.")

In [6]:
def predict_test(show_confusion_matrix=False):
    '''
        Make prediction for all images in test_x
    '''
    i = 0
    predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
    while i < len(test_x):
        j = min(i + _BATCH_SIZE, len(test_x))
        batch_xs = test_x[i:j, :]
        batch_ys = test_y[i:j, :]
        predicted_class[i:j] = sess.run(y_pred_cls, feed_dict={x: batch_xs, y: batch_ys})
        i = j

    correct = (np.argmax(test_y, axis=1) == predicted_class)
    acc = correct.mean()
    correct_numbers = correct.sum()
    print("Accuracy on Test-Set: {0:.2f}% ({1} / {2})".format(acc*100, correct_numbers, len(test_x)))

    if show_confusion_matrix is True:
        cm = confusion_matrix(y_true=np.argmax(test_y, axis=1), y_pred=predicted_class)
        for i in range(_CLASS_SIZE):
            class_name = "({}) {}".format(i, test_l[i])
            print(cm[i, :], class_name)
        class_numbers = [" ({0})".format(i) for i in range(_CLASS_SIZE)]
        print("".join(class_numbers))

    return acc

In [7]:
try:
    print("Trying to restore last checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
    saver.restore(sess, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except:
    print("Failed to restore checkpoint. Initializing variables instead.")
    sess.run(tf.global_variables_initializer())

Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from None
Failed to restore checkpoint. Initializing variables instead.


In [8]:
if _ITERATION != 0:
    train(_ITERATION)

Global Step:     10, accuracy:  12.5%, loss = 2.30 (655.6 examples/sec, 0.20 sec/batch)
Global Step:     20, accuracy:   8.6%, loss = 2.30 (680.3 examples/sec, 0.19 sec/batch)
Global Step:     30, accuracy:   9.4%, loss = 2.30 (980.3 examples/sec, 0.13 sec/batch)
Global Step:     40, accuracy:   4.7%, loss = 2.31 (965.3 examples/sec, 0.13 sec/batch)
Global Step:     50, accuracy:  12.5%, loss = 2.30 (1003.0 examples/sec, 0.13 sec/batch)
Global Step:     60, accuracy:  10.9%, loss = 2.30 (842.7 examples/sec, 0.15 sec/batch)
Global Step:     70, accuracy:  10.2%, loss = 2.30 (980.3 examples/sec, 0.13 sec/batch)
Global Step:     80, accuracy:  14.1%, loss = 2.30 (937.5 examples/sec, 0.14 sec/batch)
Global Step:     90, accuracy:  15.6%, loss = 2.30 (924.8 examples/sec, 0.14 sec/batch)
Global Step:    100, accuracy:  11.7%, loss = 2.30 (981.4 examples/sec, 0.13 sec/batch)
Accuracy on Test-Set: 10.16% (1016 / 10000)
Saved checkpoint.
Global Step:    110, accuracy:   7.0%, loss = 2.30 (939.4

Global Step:    890, accuracy:  48.4%, loss = 1.44 (918.4 examples/sec, 0.14 sec/batch)
Global Step:    900, accuracy:  41.4%, loss = 1.63 (981.0 examples/sec, 0.13 sec/batch)
Accuracy on Test-Set: 43.29% (4329 / 10000)
Saved checkpoint.
Global Step:    910, accuracy:  39.8%, loss = 1.59 (979.4 examples/sec, 0.13 sec/batch)
Global Step:    920, accuracy:  35.2%, loss = 1.66 (971.3 examples/sec, 0.13 sec/batch)
Global Step:    930, accuracy:  39.1%, loss = 1.53 (1010.7 examples/sec, 0.13 sec/batch)
Global Step:    940, accuracy:  40.6%, loss = 1.70 (931.2 examples/sec, 0.14 sec/batch)
Global Step:    950, accuracy:  39.1%, loss = 1.62 (935.0 examples/sec, 0.14 sec/batch)
Global Step:    960, accuracy:  37.5%, loss = 1.57 (962.7 examples/sec, 0.13 sec/batch)
Global Step:    970, accuracy:  46.9%, loss = 1.48 (869.9 examples/sec, 0.15 sec/batch)
Global Step:    980, accuracy:  45.3%, loss = 1.54 (883.4 examples/sec, 0.14 sec/batch)
Global Step:    990, accuracy:  40.6%, loss = 1.52 (938.4

Global Step:   1760, accuracy:  62.5%, loss = 1.17 (916.6 examples/sec, 0.14 sec/batch)
Global Step:   1770, accuracy:  51.6%, loss = 1.33 (866.1 examples/sec, 0.15 sec/batch)
Global Step:   1780, accuracy:  55.5%, loss = 1.29 (917.0 examples/sec, 0.14 sec/batch)
Global Step:   1790, accuracy:  49.2%, loss = 1.33 (933.4 examples/sec, 0.14 sec/batch)
Global Step:   1800, accuracy:  55.5%, loss = 1.29 (946.9 examples/sec, 0.14 sec/batch)
Accuracy on Test-Set: 51.10% (5110 / 10000)
Saved checkpoint.
Global Step:   1810, accuracy:  52.3%, loss = 1.25 (935.0 examples/sec, 0.14 sec/batch)
Global Step:   1820, accuracy:  51.6%, loss = 1.31 (945.6 examples/sec, 0.14 sec/batch)
Global Step:   1830, accuracy:  53.1%, loss = 1.25 (932.8 examples/sec, 0.14 sec/batch)
Global Step:   1840, accuracy:  64.8%, loss = 1.03 (929.8 examples/sec, 0.14 sec/batch)
Global Step:   1850, accuracy:  49.2%, loss = 1.34 (1018.9 examples/sec, 0.13 sec/batch)
Global Step:   1860, accuracy:  56.2%, loss = 1.23 (907.6

Global Step:   2630, accuracy:  53.9%, loss = 1.28 (1019.9 examples/sec, 0.13 sec/batch)
Global Step:   2640, accuracy:  57.8%, loss = 1.15 (974.7 examples/sec, 0.13 sec/batch)
Global Step:   2650, accuracy:  70.3%, loss = 1.00 (1017.0 examples/sec, 0.13 sec/batch)
Global Step:   2660, accuracy:  49.2%, loss = 1.23 (949.5 examples/sec, 0.13 sec/batch)
Global Step:   2670, accuracy:  58.6%, loss = 1.18 (895.1 examples/sec, 0.14 sec/batch)
Global Step:   2680, accuracy:  52.3%, loss = 1.29 (1018.9 examples/sec, 0.13 sec/batch)
Global Step:   2690, accuracy:  60.9%, loss = 1.08 (938.5 examples/sec, 0.14 sec/batch)
Global Step:   2700, accuracy:  68.0%, loss = 1.02 (933.6 examples/sec, 0.14 sec/batch)
Accuracy on Test-Set: 55.19% (5519 / 10000)
Saved checkpoint.
Global Step:   2710, accuracy:  58.6%, loss = 1.30 (1021.3 examples/sec, 0.13 sec/batch)
Global Step:   2720, accuracy:  63.3%, loss = 1.13 (1021.5 examples/sec, 0.13 sec/batch)
Global Step:   2730, accuracy:  62.5%, loss = 1.04 (9

Accuracy on Test-Set: 60.24% (6024 / 10000)
Saved checkpoint.
Global Step:   3510, accuracy:  66.4%, loss = 0.95 (983.6 examples/sec, 0.13 sec/batch)
Global Step:   3520, accuracy:  59.4%, loss = 1.04 (934.1 examples/sec, 0.14 sec/batch)
Global Step:   3530, accuracy:  68.8%, loss = 0.83 (892.0 examples/sec, 0.14 sec/batch)
Global Step:   3540, accuracy:  73.4%, loss = 0.91 (941.4 examples/sec, 0.14 sec/batch)
Global Step:   3550, accuracy:  70.3%, loss = 0.88 (1026.1 examples/sec, 0.12 sec/batch)
Global Step:   3560, accuracy:  71.1%, loss = 0.82 (909.6 examples/sec, 0.14 sec/batch)
Global Step:   3570, accuracy:  67.2%, loss = 1.02 (929.4 examples/sec, 0.14 sec/batch)
Global Step:   3580, accuracy:  73.4%, loss = 0.93 (941.2 examples/sec, 0.14 sec/batch)
Global Step:   3590, accuracy:  68.0%, loss = 1.07 (1018.8 examples/sec, 0.13 sec/batch)
Global Step:   3600, accuracy:  75.8%, loss = 0.81 (977.5 examples/sec, 0.13 sec/batch)
Accuracy on Test-Set: 62.92% (6292 / 10000)
Saved checkp

...

Global Step:   6120, accuracy:  82.8%, loss = 0.59 (953.5 examples/sec, 0.13 sec/batch)
Global Step:   6130, accuracy:  75.0%, loss = 0.70 (915.1 examples/sec, 0.14 sec/batch)
Global Step:   6140, accuracy:  77.3%, loss = 0.70 (951.3 examples/sec, 0.13 sec/batch)
Global Step:   6150, accuracy:  78.1%, loss = 0.65 (942.6 examples/sec, 0.14 sec/batch)
Global Step:   6160, accuracy:  75.0%, loss = 0.68 (912.6 examples/sec, 0.14 sec/batch)
Global Step:   6170, accuracy:  83.6%, loss = 0.54 (941.9 examples/sec, 0.14 sec/batch)
Global Step:   6180, accuracy:  79.7%, loss = 0.62 (941.4 examples/sec, 0.14 sec/batch)
Global Step:   6190, accuracy:  78.9%, loss = 0.63 (1017.1 examples/sec, 0.13 sec/batch)
Global Step:   6200, accuracy:  82.0%, loss = 0.50 (1022.5 examples/sec, 0.13 sec/batch)
Accuracy on Test-Set: 68.14% (6814 / 10000)
Saved checkpoint.
Global Step:   6210, accuracy:  80.5%, loss = 0.63 (940.9 examples/sec, 0.14 sec/batch)
Global Step:   6220, accuracy:  81.2%, loss = 0.61 (934.

Global Step:   7000, accuracy:  82.0%, loss = 0.55 (933.3 examples/sec, 0.14 sec/batch)
Accuracy on Test-Set: 70.57% (7057 / 10000)
Saved checkpoint.
Global Step:   7010, accuracy:  85.2%, loss = 0.55 (942.3 examples/sec, 0.14 sec/batch)
Global Step:   7020, accuracy:  82.8%, loss = 0.57 (977.9 examples/sec, 0.13 sec/batch)
Global Step:   7030, accuracy:  80.5%, loss = 0.57 (909.2 examples/sec, 0.14 sec/batch)
Global Step:   7040, accuracy:  78.9%, loss = 0.61 (917.8 examples/sec, 0.14 sec/batch)
Global Step:   7050, accuracy:  85.2%, loss = 0.54 (908.2 examples/sec, 0.14 sec/batch)
Global Step:   7060, accuracy:  82.0%, loss = 0.49 (892.4 examples/sec, 0.14 sec/batch)
Global Step:   7070, accuracy:  80.5%, loss = 0.61 (884.8 examples/sec, 0.14 sec/batch)
Global Step:   7080, accuracy:  80.5%, loss = 0.55 (939.2 examples/sec, 0.14 sec/batch)
Global Step:   7090, accuracy:  79.7%, loss = 0.67 (978.2 examples/sec, 0.13 sec/batch)
Global Step:   7100, accuracy:  84.4%, loss = 0.50 (918.9 

Global Step:   7870, accuracy:  80.5%, loss = 0.56 (934.9 examples/sec, 0.14 sec/batch)
Global Step:   7880, accuracy:  83.6%, loss = 0.47 (937.8 examples/sec, 0.14 sec/batch)
Global Step:   7890, accuracy:  85.2%, loss = 0.44 (1019.8 examples/sec, 0.13 sec/batch)
Global Step:   7900, accuracy:  84.4%, loss = 0.51 (987.3 examples/sec, 0.13 sec/batch)
Accuracy on Test-Set: 71.43% (7143 / 10000)
Saved checkpoint.
Global Step:   7910, accuracy:  80.5%, loss = 0.54 (983.2 examples/sec, 0.13 sec/batch)
Global Step:   7920, accuracy:  88.3%, loss = 0.41 (1022.0 examples/sec, 0.13 sec/batch)
Global Step:   7930, accuracy:  81.2%, loss = 0.52 (952.1 examples/sec, 0.13 sec/batch)
Global Step:   7940, accuracy:  82.8%, loss = 0.46 (1017.6 examples/sec, 0.13 sec/batch)
Global Step:   7950, accuracy:  88.3%, loss = 0.46 (910.2 examples/sec, 0.14 sec/batch)
Global Step:   7960, accuracy:  82.8%, loss = 0.45 (949.6 examples/sec, 0.13 sec/batch)
Global Step:   7970, accuracy:  87.5%, loss = 0.43 (935

Global Step:   8740, accuracy:  83.6%, loss = 0.45 (984.0 examples/sec, 0.13 sec/batch)
Global Step:   8750, accuracy:  85.9%, loss = 0.46 (943.9 examples/sec, 0.14 sec/batch)
Global Step:   8760, accuracy:  82.0%, loss = 0.50 (967.9 examples/sec, 0.13 sec/batch)
Global Step:   8770, accuracy:  85.2%, loss = 0.54 (1020.0 examples/sec, 0.13 sec/batch)
Global Step:   8780, accuracy:  81.2%, loss = 0.48 (945.4 examples/sec, 0.14 sec/batch)
Global Step:   8790, accuracy:  87.5%, loss = 0.41 (909.9 examples/sec, 0.14 sec/batch)
Global Step:   8800, accuracy:  89.8%, loss = 0.37 (960.1 examples/sec, 0.13 sec/batch)
Accuracy on Test-Set: 72.16% (7216 / 10000)
Saved checkpoint.
Global Step:   8810, accuracy:  83.6%, loss = 0.45 (971.6 examples/sec, 0.13 sec/batch)
Global Step:   8820, accuracy:  88.3%, loss = 0.44 (977.3 examples/sec, 0.13 sec/batch)
Global Step:   8830, accuracy:  87.5%, loss = 0.37 (918.9 examples/sec, 0.14 sec/batch)
Global Step:   8840, accuracy:  82.0%, loss = 0.46 (1022.

Global Step:   9610, accuracy:  89.8%, loss = 0.36 (1020.7 examples/sec, 0.13 sec/batch)
Global Step:   9620, accuracy:  86.7%, loss = 0.42 (932.6 examples/sec, 0.14 sec/batch)
Global Step:   9630, accuracy:  93.0%, loss = 0.28 (901.1 examples/sec, 0.14 sec/batch)
Global Step:   9640, accuracy:  84.4%, loss = 0.50 (887.3 examples/sec, 0.14 sec/batch)
Global Step:   9650, accuracy:  86.7%, loss = 0.36 (988.0 examples/sec, 0.13 sec/batch)
Global Step:   9660, accuracy:  84.4%, loss = 0.50 (905.7 examples/sec, 0.14 sec/batch)
Global Step:   9670, accuracy:  82.0%, loss = 0.45 (937.4 examples/sec, 0.14 sec/batch)
Global Step:   9680, accuracy:  89.1%, loss = 0.40 (1022.4 examples/sec, 0.13 sec/batch)
Global Step:   9690, accuracy:  89.1%, loss = 0.35 (1024.9 examples/sec, 0.12 sec/batch)
Global Step:   9700, accuracy:  87.5%, loss = 0.46 (1023.7 examples/sec, 0.13 sec/batch)
Accuracy on Test-Set: 72.99% (7299 / 10000)
Saved checkpoint.
Global Step:   9710, accuracy:  89.1%, loss = 0.36 (10

In [9]:
predict_test(True)

Accuracy on Test-Set: 73.65% (7365 / 10000)
[828  18  30   7  12   5   7   3  58  32] (0) airplane
[ 27 838   2   8   0   6   8   2  34  75] (1) automobile
[ 85   5 671  42  57  55  42  13  13  17] (2) bird
[ 47  12  83 517  44 171  64  23  20  19] (3) cat
[ 45   5  89  47 623  48  66  60  14   3] (4) deer
[ 27   5  68 128  24 663  26  35  13  11] (5) dog
[ 10   2  49  48  22  34 812   3  10  10] (6) frog
[ 32   1  47  45  33  80   7 732   1  22] (7) horse
[ 68  21  14   6   4   4   2   1 852  28] (8) ship
[ 42  65   7   6   1   8   6   5  31 829] (9) truck
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9)


0.73650000000000004

In [10]:
g = tf.get_default_graph()
feed = g.get_tensor_by_name('data/images:0')
fetch = g.get_tensor_by_name('output/output:0')

# Feeding 3 images through the net just for testing
feed_vals = img_train_x[0:3]
res = sess.run(fetch, feed_dict={feed:feed_vals})
np.shape(feed_vals), res.shape

((3, 32, 32, 3), (3, 10))

In [11]:
# for embedding
N = 2000
p = 10
EMB = np.zeros((N, p), dtype='float32')
for i in range(N): #Of course you could do mini-batches
    EMB[i] = sess.run(fetch, feed_dict={feed: img_train_x[i:i+1,:]})
    if (i % 500 == 0 or i < 5):
        print(i)

0
1
2
3
4
500
1000
1500


In [12]:
LOG_DIR = _SAVE_PATH
from tensorflow.contrib.tensorboard.plugins import projector
import os
# The embedding variable, which needs to be stored
# Note this must a Variable not a Tensor!
embedding_var = tf.Variable(EMB,  name='Embedding_of_output')
sess.run(embedding_var.initializer)
summary_writer = tf.summary.FileWriter(LOG_DIR)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Comment out if you don't have metadata
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# Comment out if you don't want sprites
embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.png')
embedding.sprite.single_image_dim.extend([img_train_x.shape[1], img_train_x.shape[1]])

projector.visualize_embeddings(summary_writer, config)
saver = tf.train.Saver([embedding_var])
saver.save(sess, os.path.join(LOG_DIR, 'model2.ckpt'), 1)

'/tmp/tensorboard/cifar-10/model2.ckpt-1'

In [13]:
metadata_file = open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w')
metadata_file.write('Name\tClass\n')
print(train_y[:5])
print(train_l)

print(train_l[np.argmax(train_y[0])])
print(train_l[np.argmax(train_y[1])])
print(train_l[np.argmax(train_y[2])])

for i in range(N):
    metadata_file.write('%06d\t%s\n' % (i, train_l[np.argmax(train_y[i])]))
metadata_file.close()

[[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
frog
truck
truck


In [14]:
# Taken from: https://github.com/tensorflow/tensorflow/issues/6322
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding

    Args:
      data: NxHxW[x3] tensor containing the images.

    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

sprite = images_to_sprite(img_train_x[:N])

In [15]:
import scipy
scipy.misc.imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)

In [16]:
%ls -lhl /tmp/tensorboard/cifar-10

total 85M
-rw-rw-r-- 1 jac jac  16M Sep 19 12:45 -10000.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 19 12:45 -10000.index
-rw-rw-r-- 1 jac jac 151K Sep 19 12:45 -10000.meta
-rw-rw-r-- 1 jac jac  16M Sep 19 12:44 -9600.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 19 12:44 -9600.index
-rw-rw-r-- 1 jac jac 151K Sep 19 12:44 -9600.meta
-rw-rw-r-- 1 jac jac  16M Sep 19 12:44 -9700.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 19 12:44 -9700.index
-rw-rw-r-- 1 jac jac 151K Sep 19 12:44 -9700.meta
-rw-rw-r-- 1 jac jac  16M Sep 19 12:44 -9800.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 19 12:44 -9800.index
-rw-rw-r-- 1 jac jac 151K Sep 19 12:44 -9800.meta
-rw-rw-r-- 1 jac jac  16M Sep 19 12:44 -9900.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 19 12:44 -9900.index
-rw-rw-r-- 1 jac jac 151K Sep 19 12:44 -9900.meta
-rw-rw-r-- 1 jac jac  135 Sep 19 12:45 checkpoint
-rw-rw-r-- 1 jac jac 2.7M Sep 19 12:45 events.out.tfevents.1505794570.jac-T5-SKYLAKE
-rw-rw-r-- 1 jac jac  

In [17]:
#sess.close()